In [31]:
# Importamos los modulos necesarios
import pandas as pd

import pyspark  #python package
import h2o


In [2]:
# objeto principal o la base a partir de la cual cuelga toda la funcionalidad de Apache Spark
from pyspark.sql import SparkSession

# Se trata del context básico de Spark, desde donde se crean el resto de variables 
# que maneja el framework. Sólo un SparkContext puede estar activo por JVM.
from pyspark import SparkContext

# el objeto functions del modulo sql define las funciones estándar incorporadas 
# para trabajar con (valores producidos) columnas.
import pyspark.sql.functions as func

In [3]:
# create a new spark session, que sera la base para nuestra aplicacion

spark = SparkSession.builder\
                    .appName("predict")\
                    .getOrCreate()

#spark sera el punto de entrada para la aplicacion

In [21]:
df_beneficiario = spark.read.format("csv")\
                            .option("header", "true")\
                            .option("inferSchema", "true")\
                            .load("data/data_test/test_beneficiario.csv")
df_aprovisionadores = spark.read.format("csv")\
                            .option("header", "true")\
                            .option("inferSchema", "true")\
                            .load("data/data_test/test_labels_aprovisionadores.csv")
df_pacientes_aprobados = spark.read.format("csv")\
                            .option("header", "true")\
                            .option("inferSchema", "true")\
                            .load("data/data_test/test_pacientes_aprobados.csv")
df_pacientes_rechazados = spark.read.format("csv")\
                            .option("header", "true")\
                            .option("inferSchema", "true")\
                            .load("data/data_test/test_pacientes_rechazados.csv")

In [22]:
# agregamos columna aprobado en los df_pacientes_aprobados y df_pacientes_rechazados
df_pacientes_aprobados = df_pacientes_aprobados.withColumn("aprobado", func.lit(True))
df_pacientes_rechazados = df_pacientes_rechazados.withColumn("aprobado", func.lit(False))

In [23]:
# quitamos columnas sobrantes en df_pacientes_aprobados
df_pacientes_aprobados = df_pacientes_aprobados.drop("fecha_admision","fecha_desembolso","grupo_diagnostico")

In [24]:
# damos el mismo formato de columna en df_pacientes_rechazados
df_pacientes_rechazados = df_pacientes_rechazados.withColumn("deducible", func.col("deducible").cast("double"))
df_pacientes_rechazados = df_pacientes_rechazados.withColumn("procedimiento_v", func.col("procedimiento_v").cast("double"))

In [25]:
# ordenamos el df_pacientes_rechazados en el mismo orden de df_pacientes aprobados para realizar el union
df_pacientes_rechazados = df_pacientes_rechazados.select(df_pacientes_aprobados.columns)

In [26]:
# creames el df_pacientes de la union de df_pacientes_aprobados con df_pacientes_rechazados
df_pacientes = df_pacientes_aprobados.union(df_pacientes_rechazados)

In [27]:
# a cada paciente agregamos la informacion del beneficiario
df_data = df_pacientes.join(df_beneficiario, on=["cod_beneficiario"], how="left")

In [28]:
# se eliminan variables que no aportan
df_data = df_data.drop("cod_beneficiario",
                       "id_peticion",
                       "fin_peticion",
                       "inicio_peticion",
                       "meses_parte_a",
                       "meses_parte_b",
                       "muerte",
                       "nacimiento")

In [29]:
df_data.printSchema()

root
 |-- aprovisionador: string (nullable = true)
 |-- deducible: double (nullable = true)
 |-- diagnostico_a: string (nullable = true)
 |-- diagnostico_admision: string (nullable = true)
 |-- diagnostico_b: string (nullable = true)
 |-- diagnostico_c: string (nullable = true)
 |-- diagnostico_d: string (nullable = true)
 |-- diagnostico_e: string (nullable = true)
 |-- diagnostico_f: string (nullable = true)
 |-- diagnostico_g: string (nullable = true)
 |-- diagnostico_h: string (nullable = true)
 |-- diagnostico_i: string (nullable = true)
 |-- diagnostico_j: string (nullable = true)
 |-- medico_atiende: string (nullable = true)
 |-- medico_operacion: string (nullable = true)
 |-- monto_reembolso: integer (nullable = true)
 |-- otros_medicos: string (nullable = true)
 |-- procedimiento_u: string (nullable = true)
 |-- procedimiento_v: double (nullable = true)
 |-- procedimiento_w: string (nullable = true)
 |-- procedimiento_x: string (nullable = true)
 |-- procedimiento_y: double (n

In [32]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,24 mins 53 secs
H2O cluster timezone:,America/Bogota
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_josedavidarevaloespinosa_4knkwy
H2O cluster total nodes:,1
H2O cluster free memory:,3.322 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [33]:
pdf = df_data.toPandas()

In [34]:
hdf = h2o.H2OFrame(pdf)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
ls

README.md                  predict_model.ipynb
data/                      train_model.ipynb
model_identify_fraudulent


In [37]:
# cargamos el modelo para realizar la prediccion
fraud_model = h2o.load_model("model_identify_fraudulent")

In [39]:
# reasignamos variables categoricas para el entrenamiento
hdf["diagnostico_a"] = hdf["diagnostico_a"].asfactor()
hdf["deducible"] = hdf["deducible"].asfactor()
hdf["diagnostico_admision"] = hdf["diagnostico_admision"].asfactor()
hdf["diagnostico_b"] = hdf["diagnostico_b"].asfactor()
hdf["diagnostico_c"] = hdf["diagnostico_c"].asfactor()
hdf["diagnostico_d"] = hdf["diagnostico_d"].asfactor()
hdf["diagnostico_e"] = hdf["diagnostico_e"].asfactor()
hdf["diagnostico_f"] = hdf["diagnostico_f"].asfactor()
hdf["diagnostico_g"] = hdf["diagnostico_g"].asfactor()
hdf["diagnostico_h"] = hdf["diagnostico_h"].asfactor()
hdf["diagnostico_i"] = hdf["diagnostico_i"].asfactor()
hdf["procedimiento_z"] = hdf["procedimiento_z"].asfactor()
hdf["alzheimer"] = hdf["alzheimer"].asfactor()
hdf["artritis_reumatoide"] = hdf["artritis_reumatoide"].asfactor()
hdf["cancer"] = hdf["cancer"].asfactor()
hdf["ciudad"] = hdf["ciudad"].asfactor()
hdf["corazon_isquemico"] = hdf["corazon_isquemico"].asfactor()
hdf["diabetes"] = hdf["diabetes"].asfactor()
hdf["ciudad"] = hdf["ciudad"].asfactor()
hdf["enfermedad_renal"] = hdf["enfermedad_renal"].asfactor()
hdf["genero"] = hdf["genero"].asfactor()
hdf["infarto_cerebral"] = hdf["infarto_cerebral"].asfactor()
hdf["municipio"] = hdf["municipio"].asfactor()
hdf["obstruccion_pulmonar"] = hdf["obstruccion_pulmonar"].asfactor()
hdf["osteoporosis"] = hdf["osteoporosis"].asfactor()
hdf["problemas_corazon"] = hdf["problemas_corazon"].asfactor()
hdf["problemas_rinon"] = hdf["problemas_rinon"].asfactor()
hdf["raza"] = hdf["raza"].asfactor()

In [40]:
hdf["es_fraudulento"] = fraud_model.predict(hdf)

KeyboardInterrupt: 